In [ ]:
import pandas as pd

# 加载数据
calendar = pd.read_csv('calendar.csv')
sales = pd.read_csv('sales_train_validation.csv')
prices = pd.read_csv('sell_prices.csv')

# 查看维度
print("calendar:", calendar.shape)
print("sales:", sales.shape)
print("prices:", prices.shape)
 

calendar: (1969, 14)
sales: (30490, 1919)
prices: (6841121, 4)


In [2]:
# 把 d_1 到 d_1913 的列转换为行
sales_long = pd.melt(
    sales,
    id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
    var_name='d',
    value_name='sales'
)


In [3]:
# 合并 calendar 中的日期特征（包括 wday、event、snap 等）
sales_long = sales_long.merge(calendar, on='d', how='left')


In [4]:
# 合并价格 sell_prices（按 store_id, item_id, wm_yr_wk）
sales_long = sales_long.merge(
    prices,
    on=['store_id', 'item_id', 'wm_yr_wk'],
    how='left'
)


In [5]:
print(sales_long.columns)
print(sales_long.head())


Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'sales', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'sell_price'],
      dtype='object')
                              id        item_id    dept_id   cat_id store_id  \
0  HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1   
1  HOBBIES_1_002_CA_1_validation  HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1   
2  HOBBIES_1_003_CA_1_validation  HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1   
3  HOBBIES_1_004_CA_1_validation  HOBBIES_1_004  HOBBIES_1  HOBBIES     CA_1   
4  HOBBIES_1_005_CA_1_validation  HOBBIES_1_005  HOBBIES_1  HOBBIES     CA_1   

  state_id    d  sales        date  wm_yr_wk  ... month  year  event_name_1  \
0       CA  d_1      0  2011-01-29     11101  ...     1  2011           NaN   
1       CA  d_1      0  2011-01-29     11101  ...     1  2011      

In [6]:
# 确保按商品和时间排序
sales_long = sales_long.sort_values(by=['id', 'date'])

# 滞后特征
for lag in [1, 7, 28]:
    sales_long[f'lag_{lag}'] = sales_long.groupby('id')['sales'].shift(lag)

# 滚动统计特征（rolling mean / std / max）
sales_long['rolling_mean_7'] = sales_long.groupby('id')['sales'].shift(1).rolling(7).mean().reset_index(0, drop=True)
sales_long['rolling_std_7'] = sales_long.groupby('id')['sales'].shift(1).rolling(7).std().reset_index(0, drop=True)
sales_long['rolling_mean_28'] = sales_long.groupby('id')['sales'].shift(1).rolling(28).mean().reset_index(0, drop=True)
sales_long['rolling_max_28'] = sales_long.groupby('id')['sales'].shift(1).rolling(28).max().reset_index(0, drop=True)

# 去除因 shift/rolling 产生的缺失行
sales_long = sales_long.dropna().reset_index(drop=True)


In [7]:
sales_long.to_csv("m5_features_with_lag.csv", index=False)
